In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
"""
Copy all the files to a local directory, because I don't have the right permissions on RDSF

"""
import os
import yaml
import shutil
import pathlib
from tqdm import tqdm

# Create an output file directory
battery_dir = "data/battery_dbs/"
if not os.path.isdir(battery_dir):
    os.makedirs(battery_dir)

# Find all the files
with open("config.yaml", "r") as stream:
    dirname = "/mnt/z/" + yaml.safe_load(stream)["smartwatch_dbs_dir"]
dirname = pathlib.Path(dirname)
assert dirname.exists()

# Recurse into all "Week X" directories, extracting all .db files
source_files = [file for file in dirname.glob("Week*/**/*.db")]
dest_files = [os.path.join(battery_dir, file.name) for file in source_files]

# Check that they don't all exist
if not all([os.path.isfile(dest) for dest in dest_files]):
    for source, dest in tqdm(zip(source_files, dest_files), total=len(source_files)):
        assert not os.path.isfile(dest), f"Duplicate name {source}"
        shutil.copyfile(source, dest)

In [ ]:
"""
Create a csv of p_id, timestamp and battery level

"""
import sqlite3
import pandas as pd


dfs = []

for path in tqdm(dest_files):
    # Get the p_id from the path
    path = pathlib.Path(path)
    p_id = path.name.split("_")[1]

    # Open the db
    conn = sqlite3.connect(path, uri=True)

    # Get the battery level
    try:
        df = pd.read_sql_query("SELECT * FROM Event;", conn)

    except pd.io.sql.DatabaseError as e:
        print(f"{path} failed with {e}")
        continue

    # Only keep battery level rows
    df = df[df["eventdesc"].str.startswith("B")]

    # Add a new column for timestamp
    df["Datetime"] = pd.to_datetime(
        df["eventdate"] + " " + df["eventtime"], format="%Y-%m-%d %H:%M:%S"
    )
    df["battery_lvl"] = df["eventdesc"].str.split(" ").str[-1].str[:-1].astype(int)

    df["p_id"] = p_id

    dfs.append(df[["p_id", "Datetime", "battery_lvl"]])

    conn.close()

battery_level_df = pd.concat(dfs)
battery_level_df.head()

In [ ]:
import matplotlib.pyplot as plt

fig, axes = plt.subplots(2, 3, figsize=(10, 4))

for pid, axis in zip(battery_level_df["p_id"].unique(), axes.flat):
    slice_ = battery_level_df[battery_level_df["p_id"] == pid].sort_values("Datetime")
    axis.plot(slice_["Datetime"], slice_["battery_lvl"], "-")

fig.suptitle("Battery Level")

fig.autofmt_xdate()
fig.tight_layout()

imgdir = "img/battery_level"
if not os.path.isdir(imgdir):
    os.makedirs(imgdir)
fig.savefig(f"{imgdir}/battery_level.png")

In [ ]:
"""
Find the start day of each participant

"""
from ema import read

battery_df = battery_level_df.copy()
battery_df["p_id"] = battery_df["p_id"].astype(int)
battery_df = battery_df.set_index(battery_df["Datetime"]).drop(columns=["Datetime"])

battery_df = read.add_timedelta(battery_df)
len(battery_df)

In [ ]:
"""
Remove negative delta and deltas above 7

"""
battery_df = battery_df[battery_df["delta"].dt.days >= 1]
battery_df = battery_df[battery_df["delta"].dt.days <= 7]
len(battery_df)

In [ ]:
"""
Plot battery level against delta

"""
from itertools import islice
import numpy as np

fig, axes = plt.subplots(3, 5, figsize=(10, 4), sharex=True, sharey=True)

for axis, (p_id, grp) in zip(
    axes.flat, islice(battery_df.groupby("p_id"), len(axes.flat))
):
    axis.plot(grp["delta"].dt.total_seconds(), grp["battery_lvl"])
    axis.set_title(p_id)

for axis in axes[:, 0]:
    axis.set_ylabel("Battery %")
for axis in axes[1]:
    axis.set_xlabel("Day")

# Convert x labels to days
axis.set_xticks(np.arange(1, 8) * 24 * 3600, range(1, 8))

fig.tight_layout()
fig.savefig(f"{imgdir}/battery_level_vs_delta.png")

In [ ]:
"""
Find the last recording day for each participant

"""
max_delta = battery_df.groupby("p_id")["delta"].max()

fig, axis = plt.subplots()
axis.hist(max_delta.dt.days, bins=range(8))
axis.set_xlabel("Day of last battery reading")
axis.set_ylabel("N participants")

fig.tight_layout()
fig.savefig(f"{imgdir}/last_battery_reading.png")

In [ ]:
"""
Find the last entry day of each participant

"""
from ema import clean

# Read in meal entries
meal_df = clean.cleaned_smartwatch(keep_catchups=False)

# Remove the non-responses
meal_df = meal_df[meal_df["meal_type"] != "No response"]

# Find the last entry for each participant
last_entry = meal_df.groupby("p_id")["delta"].max()

In [ ]:
fig, axis = plt.subplots()

hist_kw = {"bins": range(8), "histtype": "step", "linewidth": 2, "alpha": 0.8}
axis.hist(max_delta.dt.days, **hist_kw, color="r", label="Last battery reading")
axis.hist(
    last_entry.dt.days, **hist_kw, linestyle="--", color="k", label="Last meal entry"
)
axis.set_xlabel("Day")
axis.set_ylabel("N participants")
axis.legend()
axis.set_title("Slightly fewer battery reading entries because of corrupt tables")

fig.tight_layout()
fig.savefig(f"{imgdir}/last_meal_entry.png")

In [ ]:
"""
Scatter plot showing the last meal entry vs last battery reading

"""
from scipy.stats import pearsonr

last_entry_df = (
    pd.DataFrame(max_delta)
    .merge(last_entry, on="p_id", how="left")
    .rename(columns={"delta_x": "last_battery", "delta_y": "last_entry"})
)

fig, axis = plt.subplots()
axis.scatter(
    last_entry_df["last_battery"].dt.total_seconds(),
    last_entry_df["last_entry"].dt.total_seconds(),
)

# Fit and plot a line
coefs = np.polyfit(
    last_entry_df["last_battery"].dt.total_seconds(),
    last_entry_df["last_entry"].dt.total_seconds(),
    1,
)
x_values = np.linspace(0, 8.5) * 24 * 3600
y_values = np.poly1d(coefs)(x_values)
axis.plot(x_values, y_values, "--", color="k")

axis.set_xticks(np.arange(8) * 24 * 3600, range(8))
axis.set_yticks(np.arange(8) * 24 * 3600, range(8))

axis.set_xlim(0, 8.5 * 24 * 3600)
axis.set_ylim(0, 8.5 * 24 * 3600)

axis.set_xlabel("Last Battery Reading (day)")
axis.set_ylabel("Last Meal Entry (day)")

# Calculate the Pearson correlation coefficient and the p-value
correlation, p_value = pearsonr(
    last_entry_df["last_battery"].dt.total_seconds(),
    last_entry_df["last_entry"].dt.total_seconds(),
)

fig.suptitle(f"Correlation {correlation:.3f}")
axis.set_title(f"p-value {p_value:.3f}", fontsize="small")

fig.tight_layout()
fig.savefig(f"{imgdir}/last_meal_vs_battery.png")

In [ ]:
"""
Plot the battery level against final day

"""
fig, axis = plt.subplots()

deltas = []
battery_lvls = []
for _, group in battery_df.groupby("p_id"):
    row = group.loc[group["delta"].idxmax()]
    deltas.append(row["delta"].days)
    battery_lvls.append(row["battery_lvl"])

# Box plot for each day
for day in np.unique(deltas):
    axis.boxplot(
        [battery_lvls[i] for i, delta in enumerate(deltas) if delta == day],
        positions=[day],
        widths=0.5,
    )

axis.set_xlabel("Final Day of Recording")
axis.set_ylabel("Battery level")

axis.set_title("Battery level on final day", fontsize="small")
fig.suptitle(
    "Weak trend of lower final battery reading for participants\nwho stopped recording earlier"
)

fig.tight_layout()
fig.savefig(f"{imgdir}/final_battery_reading.png")

In [ ]:
"""
Scatter plot showing number of entries per day vs battery level on that day, colour coded by day

"""
both_df = []

for p_id in tqdm(battery_df["p_id"].unique()):
    tmp_dfs = []
    battery_this_pid = battery_df[battery_df["p_id"] == p_id]
    entries_this_pid = meal_df[meal_df["p_id"] == p_id]
    total_entries = len(entries_this_pid)

    # Find the number of days this participant recorded for
    n_days = battery_this_pid["delta"].dt.days.max()

    for i in range(n_days + 1):
        # Find the number of entries per day
        entries_on_this_day = entries_this_pid[entries_this_pid["delta"].dt.days == i]
        battery_on_this_day = battery_this_pid[battery_this_pid["delta"].dt.days == i]
        n_entries = len(entries_on_this_day)

        # Find the average battery level per day
        avg_battery_lvl = battery_on_this_day["battery_lvl"].mean()

        tmp_dfs.append(
            pd.DataFrame(
                {
                    "p_id": [p_id],
                    "day": [i],
                    "avg_battery_lvl": [avg_battery_lvl],
                    "n_entries": [n_entries],
                }
            )
        )

    both_df.append(pd.concat(tmp_dfs))

both_df = pd.concat(both_df)
both_df.head()

In [ ]:
"""
Scatter plot of avg battery level vs n entries, colour coded by day

"""
from matplotlib.colors import ListedColormap

cmap = plt.get_cmap("tab10")
cmap = ListedColormap(cmap(range(cmap.N))[:8])

fig, axis = plt.subplots()

scatter_params = {"alpha": 0.5, "cmap": cmap, "s": 25}
axis.scatter(
    both_df["n_entries"], both_df["avg_battery_lvl"], c=both_df["day"], **scatter_params
)
axis.set_xlabel("Number of entries")
axis.set_ylabel("Average battery level")

fig.colorbar(axis.collections[0], label="Day")

fig.tight_layout()
fig.savefig(f"{imgdir}/avg_battery_vs_n_entries.png")

In [ ]:
"""
Repeat removing 0 entry days

"""
fig, axis = plt.subplots()

both_df_no0 = both_df[both_df["n_entries"] > 0]

axis.scatter(
    both_df_no0["n_entries"],
    both_df_no0["avg_battery_lvl"],
    c=both_df_no0["day"],
    **scatter_params,
    vmin=0,
)
axis.set_xlabel("Number of entries")
axis.set_ylabel("Average battery level")

fig.colorbar(axis.collections[0], label="Day")

fig.tight_layout()
fig.savefig(f"{imgdir}/avg_battery_vs_n_entries_zero_removed.png")

In [ ]:
"""
Average over participants and days

"""
grouped_df = both_df.groupby("day").mean()
err_df = both_df.groupby("day").std()
fig, axis = plt.subplots()
axis.errorbar(
    grouped_df["n_entries"],
    grouped_df["avg_battery_lvl"],
    xerr=err_df["n_entries"],
    yerr=err_df["avg_battery_lvl"],
    fmt="none",
    c="k",
    alpha=0.3,
)

scatter_params["alpha"] = 1
axis.scatter(
    grouped_df["n_entries"],
    grouped_df["avg_battery_lvl"],
    c=grouped_df.index,
    **scatter_params
)
fig.colorbar(axis.collections[-1], label="Day")

axis.set_xlabel("Avg entries per day")
axis.set_ylabel("Avg battery level")

fig.tight_layout()
fig.savefig(f"{imgdir}/avg_battery_vs_n_entries_avgd.png")

In [ ]:
"""
Add demographic information to the df

"""

demographic_df = read.full_questionnaire()
demographic_df = demographic_df[demographic_df["respondent_status"] == 1]

demographic_df = both_df.merge(
    demographic_df[["respondent_sex", "respondent_ethnicity", "age_dob", "residents_id"]],
    left_on="p_id",
    right_on="residents_id",
    how="left",
)

# Make it easier by just reading in the cleaned smartwatch data, and using that to tell us about the ramadan status
cleaned_sw = clean.cleaned_smartwatch(keep_catchups=False)

demographic_df = demographic_df.merge(cleaned_sw[["p_id", "all_in_ramadan", "any_in_ramadan"]], on="p_id")
demographic_df.head()

In [ ]:
fig, axis = plt.subplots()

axis.plot(
    demographic_df[demographic_df["respondent_sex"] == 1]["n_entries"],
    demographic_df[demographic_df["respondent_sex"] == 1]["avg_battery_lvl"],
    marker="o",
    linestyle="None",
    color="green",
    alpha=0.5,
    label="Male",
)
axis.plot(
    demographic_df[demographic_df["respondent_sex"] == 2]["n_entries"],
    demographic_df[demographic_df["respondent_sex"] == 2]["avg_battery_lvl"],
    marker="o",
    linestyle="None",
    color="purple",
    alpha=0.5,
    label="Female",
)

axis.set_xlabel("Number of entries")
axis.set_ylabel("Average battery level")

axis.legend()

fig.tight_layout()
fig.savefig(f"{imgdir}/avg_battery_sex_separated.png")

In [ ]:
fig, axis = plt.subplots()

for label in demographic_df["respondent_ethnicity"].unique():
    axis.plot(
        demographic_df[demographic_df["respondent_ethnicity"] == label]["n_entries"],
        demographic_df[demographic_df["respondent_ethnicity"] == label][
            "avg_battery_lvl"
        ],
        marker="o",
        linestyle="None",
        alpha=0.5,
        label=f"Ethnicity {int(label)}",
    )

axis.set_xlabel("Number of entries")
axis.set_ylabel("Average battery level")

axis.legend()

fig.tight_layout()
fig.savefig(f"{imgdir}/avg_battery_ethnicity_separated.png")

In [ ]:
"""
Split by Ramadan status

"""

fig, axis = plt.subplots()

axis.plot(
    demographic_df[~demographic_df["all_in_ramadan"]]["n_entries"],
    demographic_df[~demographic_df["all_in_ramadan"]]["avg_battery_lvl"],
    marker="o",
    linestyle="None",
    color="k",
    alpha=0.5,
    label="Outside Ramadan",
)
axis.plot(
    demographic_df[demographic_df["all_in_ramadan"]]["n_entries"],
    demographic_df[demographic_df["all_in_ramadan"]]["avg_battery_lvl"],
    marker="o",
    linestyle="None",
    color="red",
    alpha=0.5,
    label="In Ramadan",
)

axis.set_xlabel("Number of entries")
axis.set_ylabel("Average battery level")

axis.legend()

fig.tight_layout()
fig.savefig(f"{imgdir}/avg_battery_ramadan_separated.png")

In [ ]:
"""
Find how many charge/de-charge periods there are

"""
tmp_df = read.full_questionnaire()
tmp_df = tmp_df[tmp_df["respondent_status"] == 1]

battery_df = battery_df.merge(
    tmp_df[
        ["respondent_sex", "respondent_ethnicity", "age_dob", "residents_id"]
    ],
    left_on="p_id",
    right_on="residents_id",
    how="left",
)

# Make it easier by just reading in the cleaned smartwatch data, and using that to tell us about the ramadan status
cleaned_sw = clean.cleaned_smartwatch(keep_catchups=False)
demographic_df = battery_df.merge(
    cleaned_sw[["p_id", "all_in_ramadan", "any_in_ramadan"]], on="p_id"
)
battery_df.head()

In [ ]:
"""
Plot some time series of battery level, showing numbers of charge/de-charge periods

"""

charges = {}
de_charges = {}
for p_id, group in battery_df.groupby("p_id"):
    assert group["delta"].is_monotonic_increasing

    # Find the sign for the battery level diff
    diff_sign = np.sign(group["battery_lvl"].diff()).replace(0, pd.NA).infer_objects(copy=False)

    # Create a new series where the sign changes, ignoring zero differences
    sign_change = (diff_sign.diff().ne(0) & diff_sign.ne(0)).cumsum()

    # Count the number of charge and de-charge periods
    charges[p_id] = (diff_sign > 0).groupby(sign_change).sum().astype(bool).sum()
    de_charges[p_id] = (diff_sign < 0).groupby(sign_change).sum().astype(bool).sum()

In [ ]:
fig, axes = plt.subplots(3, 5, figsize=(10, 4), sharex=True, sharey=True)

for axis, (p_id, grp) in zip(
    axes.flat, islice(battery_df.groupby("p_id"), len(axes.flat))
):
    axis.plot(grp["delta"].dt.total_seconds(), grp["battery_lvl"])
    axis.set_title(f"{p_id}: {charges[p_id]}, {de_charges[p_id]}")

for axis in axes[:, 0]:
    axis.set_ylabel("Battery %")
for axis in axes[1]:
    axis.set_xlabel("Day")

# Convert x labels to days
axis.set_xticks(np.arange(1, 8) * 24 * 3600, range(1, 8))

fig.tight_layout()
fig.savefig(f"{imgdir}/battery_level_n_periods.png")